# Enhanced Mask Example

B.J. Rauscher<br>
NASA/GSFC

## 1 INTRODUCTION

This notebook shows how to get the best possible background suppression using NSClean. It assumes that the observer is able to specify which pixels can be treated as background using a highly-optimized mask. This is how we recommend using NSClean.

In [1]:
# Configuration needs to come first
import os
os.environ['MKL_NUM_THREADS'] = '8'   # Enable multi-threading
os.environ['NSCLEAN_USE_CUPY'] = 'YES' # Use GPU. If you change this, you will
                                         #   need to restart python. In practice,
                                         #   I have found that most of the time
                                         #   the GPU is no faster than using CPUs.

# Import the appropriate numerical libraries
if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
    import cupy as cp
    import numpy as np
else:
    import numpy as cp
    import numpy as np

# Other imports
from astropy.io import fits
from time import perf_counter as t # Used for benchmarking
from glob import glob

# It will be handy to have a stack
S = [] # Used for benchmarking

# NSClean specific
import nsclean as nc

## 1 Process some NRS1 Files

In [2]:
# Make a list of input files
files = glob('/local/data/home/brausche/data1/JWST/Birkmann_IFU'+\
             '/1270_rate/**/*_nrs1_rate.fits', recursive=True)

The next cell executes only in MASK mode. It loads the mask from a file and configures the model fitting parameters.

In this case, the mask is stored in a UINT8 FITS file. Pixels =0 are ignored. Pixels =1 are used for modeling the background. NSClean requires that the mask be passed as a boolean array.

The three model parameters are the boolean array that contains the mask, the number of Fourier vectors to fit, and the number of Fourier vectors (within the 8192+512) that are rolled off by the apodizing filter. To enable the background modeling to be done by a consumer grade GPU, the Fourier vectors are projected out in groups of 128 at a time. The number of Fourier vectors to fit is required to be an integral multiple of 128. The apodizer can roll over over however many vectors are desired. Here we arbitrarily choose 1024.

In [3]:
# Load this channel's mask
with fits.open('/local/data/home/brausche/data1/JWST/Birkmann_IFU/nrs1_ifu_mask_thorough.fits') as hdul:
    M = cp.array(hdul[0].data, dtype=cp.bool_)

In [4]:
# Instantiate an NSClean object
cleaner = nc.NSClean('NRS1', M)

In [5]:
S.append(t()) # Put start time on stack
for i in np.arange(len(files)):
    
    # Speed up for development by doing just one
    if i != 1: continue
    
    # Download FITS file
    hdul = fits.open(files[i])
    H0 = hdul[0].header
    H1 = hdul[1].header
    D1 = cp.array(hdul[1].data)
    hdul.close()
    
    # Clean it
    D1 = cleaner.clean(D1, buff=True)
    
    # Save it to FITS
    H0['comment'] = 'Processed by NSClean Rev. '+nc.__version__
    if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
        hdul = fits.HDUList([fits.PrimaryHDU(header=H0),
                   fits.ImageHDU(cp.asnumpy(D1), header=H1, name='SCI')])
    else:
        hdul = fits.HDUList([fits.PrimaryHDU(header=H0),
                   fits.ImageHDU(D1, header=H1, name='SCI')])
    hdul.writeto('/local/data/home/brausche/tmp/'+\
                     nc.chsuf(os.path.basename(files[i]), '.cln_mask.fits'),
                        overwrite=True)
    hdul.close()
    
print('Execution time (s) = ', t()-S.pop())

Execution time (s) =  6.8839761689305305


## Same but for NRS2

In [6]:
# Make a list of input files
files = glob('/local/data/home/brausche/data1/JWST/Birkmann_IFU'+\
                 '/1270_rate/**/*_nrs2_rate.fits', recursive=True)

In [7]:
# Load this channel's mask
with fits.open('/local/data/home/brausche/data1/JWST/Birkmann_IFU/nrs2_ifu_mask_thorough.fits') as hdul:
    M = cp.array(hdul[0].data, dtype=cp.bool_)

In [8]:
# Instantiate an NSClean object
cleaner = nc.NSClean('NRS2', M)

In [9]:
stack = []
stack.append(t())
for i in np.arange(len(files)):
    
    # Speed up for development by doing just one
    if i != 1: continue
    
    # Download FITS file
    hdul = fits.open(files[i])
    H0 = hdul[0].header
    H1 = hdul[1].header
    D1 = cp.array(hdul[1].data)
    hdul.close()
    
    # Clean it
    D1 = cleaner.clean(D1, buff=True)
    
    # Save it
    H0['comment'] = 'Processed by NSClean Rev. '+nc.__version__
    if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
        hdul = fits.HDUList([fits.PrimaryHDU(header=H0),
                   fits.ImageHDU(cp.asnumpy(D1), header=H1, name='SCI')])
    else:
        hdul = fits.HDUList([fits.PrimaryHDU(header=H0),
                   fits.ImageHDU(D1, header=H1, name='SCI')])
    hdul.writeto('/local/data/home/brausche/tmp/'+nc.chsuf(os.path.basename(files[i]), '.cln_mask.fits'),
                overwrite=True)
    hdul.close()
    
print('Execution time (s) = ', t()-stack.pop())

Execution time (s) =  6.5168963968753815
